In [1]:
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from lohrasb.best_estimator import BaseModel
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer
    )
from category_encoders import OrdinalEncoder
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score)
from sklearn.metrics import f1_score, make_scorer
from xgboost import *
import optuna
from lohrasb import logger


2023-07-02 18:31:54,188 :: matplotlib :: matplotlib data path: /Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/matplotlib/mpl-data
2023-07-02 18:31:54,194 :: matplotlib :: CONFIGDIR=/Users/hjavedani/.matplotlib
2023-07-02 18:31:54,197 :: matplotlib :: interactive is False
2023-07-02 18:31:54,198 :: matplotlib :: platform is darwin
2023-07-02 18:31:54,210 :: graphviz._tools :: deprecate positional args: graphviz.backend.piping.pipe(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2023-07-02 18:31:54,212 :: graphviz._tools :: deprecate positional args: graphviz.backend.rendering.render(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2023-07-02 18:31:54,215 :: graphviz._tools :: deprecate positional args: graphviz.backend.unflattening.unflatten(['stagger', 'fanout', 'chain', 'encoding'])
2023-07-02 18:31:54,217 :: graphviz._tools :: deprecate positional args: graphviz.backend.viewing.view(['quiet'])
2023-07-02 18:31:54,221 :: graphviz._tools :: deprecate positio

#### Example: Use Adult Data Set (a classification problem)
  
https://archive.ics.uci.edu/ml/datasets/Adult

#### Part 1: Use BestModel in sklearn pipeline


In [2]:
urldata= "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
# column names
col_names=["age", "workclass", "fnlwgt" , "education" ,"education-num",
"marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week",
"native-country","label"
]
# read data
data = pd.read_csv(urldata,header=None,names=col_names,sep=',')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### Define labels


In [3]:
data.loc[data['label']=='<=50K','label']=0
data.loc[data['label']==' <=50K','label']=0

data.loc[data['label']=='>50K','label']=1
data.loc[data['label']==' >50K','label']=1

data['label']=data['label'].astype(int)

#### Train test split

In [4]:
X = data.loc[:, data.columns != "label"]
y = data.loc[:, data.columns == "label"]


X_train, X_test, y_train, y_test =train_test_split(X, y, \
     test_size=0.33, stratify=y['label'], random_state=42)


#### Find feature types for later use

In [5]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()
float_cols =  X_train.select_dtypes(include=['float']).columns.tolist()
cat_cols =  X_train.select_dtypes(include=['object']).columns.tolist()


#### Define estimator and set its arguments 


In [6]:
estimator = XGBClassifier()
estimator_params = {
        "booster": optuna.distributions.CategoricalDistribution(choices=("gbtree","dart")),
        "max_depth": optuna.distributions.IntDistribution(10, 15),
        "gamma": optuna.distributions.FloatDistribution(0.5, 1.2, log=True),
        "subsample": optuna.distributions.FloatDistribution(0.8, 1.0)
        }

    

In [7]:
kwargs = {  # params for fit method or fit_params 
            'fit_newoptuna_kwargs' :{
            'sample_weight':None,
            },
            # params for GridSearchCV 
            'newoptuna_search_kwargs' : {
            'estimator':estimator,
            'param_distributions':estimator_params,
            'scoring' :'f1',
            'verbose':3,
            'n_jobs':-1,
            'cv':KFold(2),
            }
            }


In [8]:

obj = BaseModel().optimize_by_optunasearchcv(
        kwargs=kwargs    
        )

#### Build sklearn pipeline

In [9]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model
            ('obj', obj)

 ])


#### Run Pipeline

In [10]:
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)


/Users/hjavedani/Documents/Lohrasb/lohrasb/base_classes/optimizer_bases.py:579: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  self.__newoptuna_search = OptunaSearchCV(**self.newoptuna_search_kwargs)
[I 2023-07-02 18:31:58,232] A new study created in memory with name: no-name-edda8bee-acb9-4341-a0c0-810ef1b7776d
[I 2023-07-02 18:31:58,232] Searching the best hyperparameters using 21815 samples...
[I 2023-07-02 18:32:11,826] Trial 3 finished with value: 0.6731619494967017 and parameters: {'booster': 'gbtree', 'max_depth': 11, 'gamma': 0.6286956035282129, 'subsample': 0.9261049725212485}. Best is trial 3 with value: 0.6731619494967017.
[I 2023-07-02 18:32:27,315] Trial 7 finished with value: 0.6728612624426478 and parameters: {'booster': 'dart', 'max_depth': 10, 'gamma': 0.7657748791836362, 'subsample': 0.9230292200998791}. Best is trial 3 with value: 0.6731619494967017.
[I 2023-07-02 18:32:27,455] Trial 2 finished 

#### Check performance of the pipeline

In [11]:
print('F1 score : ')
print(f1_score(y_test,y_pred))
print('Classification report : ')
print(classification_report(y_test,y_pred))
print('Confusion matrix : ')
print(confusion_matrix(y_test,y_pred))


F1 score : 
0.6814213197969543
Classification report : 
              precision    recall  f1-score   support

           0       0.89      0.92      0.91      8158
           1       0.72      0.65      0.68      2588

    accuracy                           0.85     10746
   macro avg       0.80      0.78      0.79     10746
weighted avg       0.85      0.85      0.85     10746

Confusion matrix : 
[[7499  659]
 [ 910 1678]]


#### Part 2:  Use BestModel as a standalone estimator 

In [12]:
X_train, X_test, y_train, y_test =train_test_split(X, y, \
     test_size=0.33, stratify=y['label'], random_state=42)

#### Transform features to make them ready for model input

In [13]:
transform_pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model

 ])

#### Transform X_train and X_test

In [14]:
X_train=transform_pipeline.fit_transform(X_train,y_train)
X_test=transform_pipeline.transform(X_test)


#### Train model and predict

In [15]:
obj.fit(X_train,y_train)
y_pred = obj.predict(X_test)

/Users/hjavedani/Documents/Lohrasb/lohrasb/base_classes/optimizer_bases.py:579: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  self.__newoptuna_search = OptunaSearchCV(**self.newoptuna_search_kwargs)
[I 2023-07-02 18:32:38,806] A new study created in memory with name: no-name-41c86a39-7ff6-4f76-a79f-745af30e5e6d
[I 2023-07-02 18:32:38,807] Searching the best hyperparameters using 21815 samples...
[I 2023-07-02 18:32:49,680] Trial 0 finished with value: 0.6716672852252672 and parameters: {'booster': 'gbtree', 'max_depth': 10, 'gamma': 0.5506768705403388, 'subsample': 0.8370260396883908}. Best is trial 0 with value: 0.6716672852252672.
[I 2023-07-02 18:32:50,902] Trial 2 finished with value: 0.6739886481835298 and parameters: {'booster': 'gbtree', 'max_depth': 11, 'gamma': 0.8077815768620721, 'subsample': 0.8391652116972823}. Best is trial 2 with value: 0.6739886481835298.
[I 2023-07-02 18:32:51,667] Trial 5 finishe

#### Check performance of the pipeline

In [16]:
print('F1 score : ')
print(f1_score(y_test,y_pred))
print('Classification report : ')
print(classification_report(y_test,y_pred))
print('Confusion matrix : ')
print(confusion_matrix(y_test,y_pred))

F1 score : 
0.6835236541598696
Classification report : 
              precision    recall  f1-score   support

           0       0.89      0.92      0.91      8158
           1       0.72      0.65      0.68      2588

    accuracy                           0.86     10746
   macro avg       0.81      0.78      0.79     10746
weighted avg       0.85      0.86      0.85     10746

Confusion matrix : 
[[7518  640]
 [ 912 1676]]


In [17]:
obj.get_best_estimator()

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.8077815768620721,
              gpu_id=-1, grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=11, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [18]:
obj.best_estimator

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.8077815768620721,
              gpu_id=-1, grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=11, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

#### Get fitted grid search object and its attributes

In [19]:
NewSearchObj = obj.get_optimized_object()

In [20]:
NewSearchObj

OptunaSearchCV(cv=KFold(n_splits=2, random_state=None, shuffle=False),
               estimator=XGBClassifier(base_score=None, booster=None,
                                       callbacks=None, colsample_bylevel=None,
                                       colsample_bynode=None,
                                       colsample_bytree=None,
                                       early_stopping_rounds=None,
                                       enable_categorical=False,
                                       eval_metric=None, feature_types=None,
                                       gamma=None, gpu_id=None,
                                       grow_policy=None, importance_type=Non...
                                       num_parallel_tree=None, predictor=None,
                                       random_state=None, ...),
               n_jobs=-1,
               param_distributions={'booster': CategoricalDistribution(choices=('gbtree', 'dart')),
                                    'gamma': FloatDistribution(high=1.2, log=True, low=0.5, step=None),
                                    'max_depth': IntDistribution(high=15, log=False, low=10, step=1),
                                    'subsample': FloatDistribution(high=1.0, log=False, low=0.8, step=None)},
               scoring='f1', verbose=3)